# TP2
## Analyse d’applications et Cyber-sécurité

##### Groupe 2: Adrien Chevallereau 2226047, Celina Ghoraieb-Munoz 2309332


# Partie 1


## Function Call

Pour l'appel de fonction il y a plusieurs étapes dans la methode __visit_FUNCTIONCALL__ pour modifier l'arbre à deux branche à un arbre sans brache avec de nouveaux noeuds: 

- Tout d'abord, nous avons créé les noeuds CallBegin, CallEnd et RetValue qui viennent s'ajouter à la fin de l'arbre. 
- Puis nous avons créé les différentes relation entre les noeuds avec __add_edge__, __set_call__ pour l'appel de la fonction, __set_call_exp__ qui va lier l'appel de la fonction à son Id 

Dans cette étape nous avons aussi traité les __ArgumentList__ . Il ajoute le noeud Argument à la fin d'une __ArgumentList__. 

Nous avons aussi traité le cas de la liste d'argument lors de l'appel d'une fonction.

Pour cela nous avons ajouté une clé dans le dictionnaire ctx lors d'un function call pour stocker l'appelant et lors du déplacement dans les fils de l'arguments list, on va les lier à l'appel de la funciton avec un __add__call_arg__.


In [92]:
from code_analysis import ASTReader, AST, CFG, CFGException, ASTException

reader = ASTReader()
f_path = '../part_1/functioncall/functioncall.php.ast.json'
ast_data = reader.read_ast(f_path)

class ASTtoCFGVisitor:
    def __init__(self):
        self.ast = None
        self.cfg = CFG()
        self.iNextNode = 0

    def get_new_node(self) -> int:
        self.iNextNode += 1
        return self.iNextNode

    def visit(self, ast: AST):
        self.ast = ast
        self.cfg = CFG()
        print(f"Visit AST from file {self.ast.get_filename()}")
        self.visit_ROOT()
        return self.cfg

    def visit_ROOT(self):
        ctx = {}
        entryNodeId = self.get_new_node()
        stopNodeId = self.get_new_node()
        rootAST = self.ast.get_root()
        self.cfg.set_root(entryNodeId)

        self.cfg.set_type(entryNodeId, "Entry")
        self.cfg.set_image(entryNodeId, "main")
        self.cfg.set_type(stopNodeId, "Exit")

        ctx['parent'] = entryNodeId
        ctx['scope'] = entryNodeId
        ctx['stopId'] = stopNodeId

        if self.ast.get_type(rootAST) == "Start":
            self.cfg.set_node_ptr(rootAST, entryNodeId)

        self.visit_node(rootAST, ctx)
        self.cfg.add_edge(ctx['endId'], stopNodeId)

    # chain nodes
    def visit_GENERIC(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)

        ctx["endId"] = cfg_node

        new_ctx = dict(ctx) # clone ctx
        new_ctx["parent"] = cfg_node
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
        ctx["endId"] = new_ctx["endId"]

        return cfg_node

    def visit_GENERIC_BLOCK(self, ast_node_id: int, ctx: dict):
        new_ctx = dict(ctx) # clone ctx
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
        ctx["endId"] = new_ctx["endId"]

        return None

    def visit_BINOP(self, ast_node_id: int, ctx: dict) -> int:
        #Create BinOP node
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))


        #Visit right child
        new_ctx = dict(ctx) # clone ctx
        self.visit_node(self.ast.get_children(ast_node_id)[1], new_ctx)
        right = new_ctx['endId']

        #Visit right left
        new_ctx = dict(ctx) # clone ctx
        new_ctx["parent"] = right
        self.visit_node(self.ast.get_children(ast_node_id)[0], new_ctx)
        left = new_ctx['endId']

        #Link left child with BinOp
        self.cfg.add_edge(left, cfg_node)

        ctx["endId"] = cfg_node
        return cfg_node

    def visit_FUNC_CALL(self, ast_node_id: int, ctx: dict) -> int:
         #Create CallBegin node
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)




        # Ajout du noeud CallBegin
        cfg_callbegin_node = self.get_new_node()
        self.cfg.set_type(cfg_callbegin_node, "CallBegin")
        self.cfg.set_image(cfg_callbegin_node, self.ast.get_image(ast_node_id))

        #Ajout du noeud CallEnd
        cfg_callend_node = self.get_new_node()
        self.cfg.set_type(cfg_callend_node, "CallEnd")
        self.cfg.set_image(cfg_callend_node, self.ast.get_image(ast_node_id))

        #Flèche pointillé représentant l'appel de la fonction
        self.cfg.set_call(cfg_callbegin_node, cfg_callend_node)
        


        ctx["endId"] = cfg_callbegin_node
        ctx["Callbegin"] = cfg_callbegin_node
        ctx["parent"] = cfg_node

        new_ctx = dict(ctx)
        #Visite des enfants pour les lier avant l'appel de la fonction
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
            #Si l'enfant est un Id, on le lie au CallBegin
            if self.ast.get_type(child_id) == "Id" and self.ast.get_image(child_id) == self.ast.get_image(ast_node_id):
                self.cfg.set_call_expr(cfg_callbegin_node, new_ctx["endId"])



        # Ajout du noeud RetValue
        cfg_retvalue_node = self.get_new_node()
        self.cfg.set_type(cfg_retvalue_node, "RetValue")
        #Flèche entre CallEnd et RetValue
        self.cfg.add_edge(cfg_callend_node, cfg_retvalue_node)
        
        #Flèche entre la fin des enfant et argument
        self.cfg.add_edge(new_ctx["endId"], ctx["endId"])
        ctx["endId"] = cfg_retvalue_node

    def visit_ARG_LIST(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)


        ctx["endId"] = ctx["parent"]
        ctx["parent"] = cfg_node

        new_ctx = dict(ctx)
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
            #Si la liste d'argument est lié à un appel de fonction
            if "Callbegin" in ctx and  self.cfg.get_type(ctx["Callbegin"]) == "CallBegin":
                self.cfg.add_call_arg(ctx["Callbegin"], new_ctx["endId"])



        # Ajout du noeud argument
        cfg_argument_node = self.get_new_node()
        self.cfg.set_type(cfg_argument_node, "Argument")
        ctx["endId"] = cfg_argument_node
        self.cfg.add_edge(new_ctx["endId"], ctx["endId"])




    def visit_node(self, ast_node_id: int, ctx: dict):
        cur_type = self.ast.get_type(ast_node_id)

        if cur_type is None:
            raise ASTException("Missing type in a node")

        if cur_type in ["BinOP", "RelOP", "LogicOP"]:
            self.visit_BINOP(ast_node_id, ctx)
        elif cur_type in ["Block", "Start"]:
            self.visit_GENERIC_BLOCK(ast_node_id, ctx)
        elif cur_type in ["PLACEHOLDER"]: # Node to ignore
            self.visit_passthrough(ast_node_id, ctx)
        elif cur_type in ["FunctionCall"]:
            self.visit_FUNC_CALL(ast_node_id, ctx)
        elif cur_type in ["ArgumentList"]:
            self.visit_ARG_LIST(ast_node_id, ctx)  
        else:
            self.visit_GENERIC(ast_node_id, ctx)

    def visit_passthrough(self, ast_node_id: int, ctx: dict):
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, ctx)


ast_data1 = ASTtoCFGVisitor()
ast_data1.visit(ast_data)
ast_data1.cfg.show()


Visit AST from file code.php


# IF

Nous reprenons le code précédent pour ajouter le traitement du IF. Pour cela nous avons ajouté deux fonction des traitements, une pour le IfThenElseStatement et une pour le traitement de la condition. Puis nous avons modifié la fonction BinOP pour changer l'ordre lors d'un real op et ajouter les bonnes flèches.

Ainsi dans __IfThenElseStatement__ : 
- nous avons modifié le nom en IF
- nous avons ajouté le noeud "IfEnd" à la fin du if
Puis nous parcourons les différents enfants du pour lier le EndIF au différent blocs then, elif et else. Nous traitons le premier enfant à part pour éviter les doubles flèches

Dans __visit_Condition__ :
On doit exprimer la condition avant le noeud ainsi on parcours l'enfant (une seule condition possible) et on lie la fin de l'expression de la condition au noeud condition.

Dans __visit_BinOp__ :
Nous avons distingué 2 cas: 
- un avec le binOP pour lequel nous avons affecté les deux operandes de gauche à droite et nous avons utilisé les flèches __set_opHand__ pour lier les opérandes à l'opérateur
- un avec le RelOP pour lequel nous avons affecté les deux operandes de droite à gauche et nous avons utilisé les flèches __add_call_arg__ pour lier les opérandes à l'opérateur en couleur bleu



In [93]:
from code_analysis import ASTReader, AST, CFG, CFGException, ASTException

reader = ASTReader()
f_path = '../part_1/if/if.php.ast.json'
ast_data = reader.read_ast(f_path)

class ASTtoCFGVisitor:
    def __init__(self):
        self.ast = None
        self.cfg = CFG()
        self.iNextNode = 0

    def get_new_node(self) -> int:
        self.iNextNode += 1
        return self.iNextNode

    def visit(self, ast: AST):
        self.ast = ast
        self.cfg = CFG()
        print(f"Visit AST from file {self.ast.get_filename()}")
        self.visit_ROOT()
        return self.cfg

    def visit_ROOT(self):
        ctx = {}
        entryNodeId = self.get_new_node()
        stopNodeId = self.get_new_node()
        rootAST = self.ast.get_root()
        self.cfg.set_root(entryNodeId)

        self.cfg.set_type(entryNodeId, "Entry")
        self.cfg.set_image(entryNodeId, "main")
        self.cfg.set_type(stopNodeId, "Exit")

        ctx['parent'] = entryNodeId
        ctx['scope'] = entryNodeId
        ctx['stopId'] = stopNodeId

        if self.ast.get_type(rootAST) == "Start":
            self.cfg.set_node_ptr(rootAST, entryNodeId)

        self.visit_node(rootAST, ctx)
        self.cfg.add_edge(ctx['endId'], stopNodeId)

    # chain nodes
    def visit_GENERIC(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)

        ctx["endId"] = cfg_node

        new_ctx = dict(ctx) # clone ctx
        new_ctx["parent"] = cfg_node
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
        ctx["endId"] = new_ctx["endId"]

        return cfg_node

    def visit_GENERIC_BLOCK(self, ast_node_id: int, ctx: dict):
        new_ctx = dict(ctx) # clone ctx
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
        ctx["endId"] = new_ctx["endId"]

        return None

    def visit_BINOP(self, ast_node_id: int, ctx: dict) -> int:
        #Create BinOP node
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))

        if self.ast.get_type(ast_node_id) == "BinOP":
            right_child = self.ast.get_children(ast_node_id)[1]
            left_child = self.ast.get_children(ast_node_id)[0]
        elif self.ast.get_type(ast_node_id) == "RelOP":
            right_child = self.ast.get_children(ast_node_id)[0]
            left_child = self.ast.get_children(ast_node_id)[1]
            
        #Visit right child
        new_ctx = dict(ctx) # clone ctx
        self.visit_node(right_child, new_ctx)
        right = new_ctx['endId']

        #Visit right left
        new_ctx = dict(ctx) # clone ctx
        new_ctx["parent"] = right
        self.visit_node(left_child, new_ctx)
        left = new_ctx['endId']

        #Link left child with BinOp
        self.cfg.add_edge(left, cfg_node)
        ctx["endId"] = cfg_node

        if self.cfg.get_type(cfg_node) == "RelOP":
            self.cfg.add_call_arg(cfg_node, right)
            self.cfg.add_call_arg(cfg_node, left)
        elif self.cfg.get_type(cfg_node) == "BinOP":
            self.cfg.set_op_hands(cfg_node, left, right)

        return cfg_node

    def visit_FUNC_CALL(self, ast_node_id: int, ctx: dict) -> int:
         #Create CallBegin node
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)




        # Ajout du noeud CallBegin
        cfg_callbegin_node = self.get_new_node()
        self.cfg.set_type(cfg_callbegin_node, "CallBegin")
        self.cfg.set_image(cfg_callbegin_node, self.ast.get_image(ast_node_id))

        #Ajout du noeud CallEnd
        cfg_callend_node = self.get_new_node()
        self.cfg.set_type(cfg_callend_node, "CallEnd")
        self.cfg.set_image(cfg_callend_node, self.ast.get_image(ast_node_id))

        #Flèche pointillé représentant l'appel de la fonction
        self.cfg.set_call(cfg_callbegin_node, cfg_callend_node)
        


        ctx["endId"] = cfg_callbegin_node
        ctx["parent"] = cfg_node
        ctx["Callbegin"] = cfg_callbegin_node

        new_ctx = dict(ctx)
        #Visite des enfants pour les lier avant l'appel de la fonction
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
            #Si l'enfant est un Id, on le lie au CallBegin
            if self.ast.get_type(child_id) == "Id" and self.ast.get_image(child_id) == self.ast.get_image(ast_node_id):
                self.cfg.set_call_expr(cfg_callbegin_node, new_ctx["endId"])
            #Si l'enfant est un ArgumentList, on lie les arguments de la liste au CallBegin
            if self.ast.get_type(child_id) == "ArgumentList":
                self.cfg.add_call_arg(cfg_callbegin_node, new_ctx["parent"])


        # Ajout du noeud RetValue
        cfg_retvalue_node = self.get_new_node()
        self.cfg.set_type(cfg_retvalue_node, "RetValue")
        #Flèche entre CallEnd et RetValue
        self.cfg.add_edge(cfg_callend_node, cfg_retvalue_node)

                
                         
        
        #Flèche entre la fin des enfant et argument
        self.cfg.add_edge(new_ctx["endId"], ctx["endId"])
        ctx["endId"] = cfg_retvalue_node


    def visit_ARG_LIST(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)


        ctx["parent"] = cfg_node

        new_ctx = dict(ctx)
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
            if "Callbegin" in ctx and  self.cfg.get_type(ctx["Callbegin"]) == "CallBegin":
                self.cfg.add_call_arg(ctx["Callbegin"], new_ctx["endId"])


        # Ajout du noeud argument
        cfg_argument_node = self.get_new_node()
        self.cfg.set_type(cfg_argument_node, "Argument")
        ctx["endId"] = cfg_argument_node
        self.cfg.add_edge(new_ctx["endId"], ctx["endId"])


    def visit_IFTHENELSE(self, ast_node_id: int, ctx: dict) -> int:
        #Moddification du noeud IfThenElseStatement
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, "If")
        self.cfg.add_edge(ctx["parent"], cfg_node)

        #Ajout du noeud IfEnd
        cfg_ifend_node = self.get_new_node()
        self.cfg.set_type(cfg_ifend_node, "IfEnd")
        ctx["endId"] = cfg_ifend_node

        ctx["parent"] = cfg_node

        children = self.ast.get_children(ast_node_id)
        new_ctx = dict (ctx)
        self.visit_node(children[0], new_ctx)
        ctx ["parent"] = new_ctx ["endId"]
        #Parcours des enfants
        new_ctx = dict(ctx) # clone ctx
        for child_id in children[1:]:
            scope = self.visit_node(child_id, new_ctx)
            self.cfg.add_edge(new_ctx["endId"], ctx["endId"])

    def visit_CONDITION(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))


        children = self.ast.get_children(ast_node_id)
        new_ctx = dict(ctx)

        self.visit_node(children[0], new_ctx)

        self.cfg.add_edge(new_ctx["endId"], cfg_node)

        ctx["endId"] = cfg_node

        

    def visit_node(self, ast_node_id: int, ctx: dict):
        cur_type = self.ast.get_type(ast_node_id)
        if cur_type is None:
            raise ASTException("Missing type in a node")

        if cur_type in ["BinOP", "RelOP", "LogicOP"]:
            self.visit_BINOP(ast_node_id, ctx)
        elif cur_type in ["Block", "Start"]:
            self.visit_GENERIC_BLOCK(ast_node_id, ctx)
        elif cur_type in ["PLACEHOLDER", "StatementBody"]: # Node to ignore
            self.visit_passthrough(ast_node_id, ctx)
        elif cur_type in ["FunctionCall"]:
            self.visit_FUNC_CALL(ast_node_id, ctx)
        elif cur_type in ["ArgumentList"]:
            self.visit_ARG_LIST(ast_node_id, ctx)
        elif cur_type in ["IfThenElseStatement"]:
            self.visit_IFTHENELSE(ast_node_id, ctx)
        elif cur_type in ["Condition"]:
            self.visit_CONDITION(ast_node_id, ctx)

        else:
            self.visit_GENERIC(ast_node_id, ctx)

    def visit_passthrough(self, ast_node_id: int, ctx: dict):
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, ctx)



ast_data1 = ASTtoCFGVisitor()
ast_data1.visit(ast_data)
ast_data1.cfg.show()


Visit AST from file code.php


# While sans le break

Nous avons utilisé le code précédents en le modifiant __visitBINOP__ et en ajoutant __visitWHILE__ pour traiter le cas while sans break et continue.
- Modification de __visit_BinOp__ pour prendre en compte les BinOP autre que "=", ainsi nous les traitons comme le cas de "RelOp" et nous ajoutons "LogicOp" au dernier cas (qui devient donc le cas "Else")
- création de __visit_WHILE__ pour traiter le cas du while sans break et continue. Nous avons ajouté le noeud "WhileEnd" à la fin du while et nous avons lié le noeud "WhileEnd" au noeud "Condition" pour le cas où la condition est fausse.
Puis nous avons lié la fin du bloc à While.


In [94]:
from code_analysis import ASTReader, AST, CFG, CFGException, ASTException

reader = ASTReader()
f_path = '../part_1/while/while.php.ast.json'
ast_data = reader.read_ast(f_path)

class ASTtoCFGVisitor:
    def __init__(self):
        self.ast = None
        self.cfg = CFG()
        self.iNextNode = 0

    def get_new_node(self) -> int:
        self.iNextNode += 1
        return self.iNextNode

    def visit(self, ast: AST):
        self.ast = ast
        self.cfg = CFG()
        print(f"Visit AST from file {self.ast.get_filename()}")
        self.visit_ROOT()
        return self.cfg

    def visit_ROOT(self):
        ctx = {}
        entryNodeId = self.get_new_node()
        stopNodeId = self.get_new_node()
        rootAST = self.ast.get_root()
        self.cfg.set_root(entryNodeId)

        self.cfg.set_type(entryNodeId, "Entry")
        self.cfg.set_image(entryNodeId, "main")
        self.cfg.set_type(stopNodeId, "Exit")

        ctx['parent'] = entryNodeId
        ctx['scope'] = entryNodeId
        ctx['stopId'] = stopNodeId

        if self.ast.get_type(rootAST) == "Start":
            self.cfg.set_node_ptr(rootAST, entryNodeId)

        self.visit_node(rootAST, ctx)
        self.cfg.add_edge(ctx['endId'], stopNodeId)

    # chain nodes
    def visit_GENERIC(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)

        ctx["endId"] = cfg_node

        new_ctx = dict(ctx) # clone ctx
        new_ctx["parent"] = cfg_node
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
        ctx["endId"] = new_ctx["endId"]

        return cfg_node

    def visit_GENERIC_BLOCK(self, ast_node_id: int, ctx: dict):
        new_ctx = dict(ctx) # clone ctx
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
        ctx["endId"] = new_ctx["endId"]

        return None

    def visit_BINOP(self, ast_node_id: int, ctx: dict) -> int:
        #Create BinOP node
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))

        if self.ast.get_type(ast_node_id) == "BinOP" and self.ast.get_image(ast_node_id) == "=":
            right_child = self.ast.get_children(ast_node_id)[1]
            left_child = self.ast.get_children(ast_node_id)[0]
        else:
            right_child = self.ast.get_children(ast_node_id)[0]
            left_child = self.ast.get_children(ast_node_id)[1]
            
        #Visit right child
        new_ctx = dict(ctx) # clone ctx
        self.visit_node(right_child, new_ctx)
        right = new_ctx['endId']

        #Visit right left
        new_ctx = dict(ctx) # clone ctx
        new_ctx["parent"] = right
        self.visit_node(left_child, new_ctx)
        left = new_ctx['endId']

        #Link left child with BinOp
        self.cfg.add_edge(left, cfg_node)
        ctx["endId"] = cfg_node

        
        if self.cfg.get_type(cfg_node) == "BinOP" and self.ast.get_image(ast_node_id) == "=":
            self.cfg.set_op_hands(cfg_node, left, right)
        else:
            self.cfg.add_call_arg(cfg_node, right)
            self.cfg.add_call_arg(cfg_node, left)

        return cfg_node

    def visit_FUNC_CALL(self, ast_node_id: int, ctx: dict) -> int:
         #Create CallBegin node
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)




        # Ajout du noeud CallBegin
        cfg_callbegin_node = self.get_new_node()
        self.cfg.set_type(cfg_callbegin_node, "CallBegin")
        self.cfg.set_image(cfg_callbegin_node, self.ast.get_image(ast_node_id))

        #Ajout du noeud CallEnd
        cfg_callend_node = self.get_new_node()
        self.cfg.set_type(cfg_callend_node, "CallEnd")
        self.cfg.set_image(cfg_callend_node, self.ast.get_image(ast_node_id))

        #Flèche pointillé représentant l'appel de la fonction
        self.cfg.set_call(cfg_callbegin_node, cfg_callend_node)
        


        ctx["endId"] = cfg_callbegin_node
        ctx["parent"] = cfg_node
        ctx["Callbegin"] = cfg_callbegin_node

        new_ctx = dict(ctx)
        #Visite des enfants pour les lier avant l'appel de la fonction
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
            #Si l'enfant est un Id, on le lie au CallBegin
            if self.ast.get_type(child_id) == "Id" and self.ast.get_image(child_id) == self.ast.get_image(ast_node_id):
                self.cfg.set_call_expr(cfg_callbegin_node, new_ctx["endId"])



        # Ajout du noeud RetValue
        cfg_retvalue_node = self.get_new_node()
        self.cfg.set_type(cfg_retvalue_node, "RetValue")
        #Flèche entre CallEnd et RetValue
        self.cfg.add_edge(cfg_callend_node, cfg_retvalue_node)

                
                         
        
        #Flèche entre la fin des enfant et argument
        self.cfg.add_edge(new_ctx["endId"], ctx["endId"])
        ctx["endId"] = cfg_retvalue_node


    def visit_ARG_LIST(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)


        ctx["parent"] = cfg_node

        new_ctx = dict(ctx)
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
            if "Callbegin" in ctx and  self.cfg.get_type(ctx["Callbegin"]) == "CallBegin":
                self.cfg.add_call_arg(ctx["Callbegin"], new_ctx["endId"])


        # Ajout du noeud argument
        cfg_argument_node = self.get_new_node()
        self.cfg.set_type(cfg_argument_node, "Argument")
        ctx["endId"] = cfg_argument_node
        self.cfg.add_edge(new_ctx["endId"], ctx["endId"])


    def visit_IFTHENELSE(self, ast_node_id: int, ctx: dict) -> int:
        #Moddification du noeud IfThenElseStatement
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, "If")
        self.cfg.add_edge(ctx["parent"], cfg_node)

        #Ajout du noeud IfEnd
        cfg_ifend_node = self.get_new_node()
        self.cfg.set_type(cfg_ifend_node, "IfEnd")
        ctx["endId"] = cfg_ifend_node

        ctx["parent"] = cfg_node

        children = self.ast.get_children(ast_node_id)
        new_ctx = dict (ctx)
        self.visit_node(children[0], new_ctx)
        ctx ["parent"] = new_ctx ["endId"]
        #Parcours des enfants
        new_ctx = dict(ctx) # clone ctx
        for child_id in children[1:]:
            scope = self.visit_node(child_id, new_ctx)
            self.cfg.add_edge(new_ctx["endId"], ctx["endId"])

    def visit_CONDITION(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))


        children = self.ast.get_children(ast_node_id)
        new_ctx = dict(ctx)

        self.visit_node(children[0], new_ctx)

        self.cfg.add_edge(new_ctx["endId"], cfg_node)

        ctx["endId"] = cfg_node

    def visit_WHILE(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)

        #Ajout du noeud WhileEnd
        cfg_whileend_node = self.get_new_node()
        self.cfg.set_type(cfg_whileend_node, "WhileEnd")
        ctx["endId"] = cfg_whileend_node

        ctx["parent"] = cfg_node

        new_ctx = dict(ctx)
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
            # On lie la condition à WhileEnd
            if self.ast.get_type(child_id) == "Condition":
                self.cfg.add_edge(new_ctx["endId"], ctx["endId"])

        #Fin du block on le lie donc à While
        self.cfg.add_edge(new_ctx["endId"], cfg_node)


        

    def visit_node(self, ast_node_id: int, ctx: dict):
        cur_type = self.ast.get_type(ast_node_id)
        if cur_type is None:
            raise ASTException("Missing type in a node")

        if cur_type in ["BinOP", "RelOP", "LogicOP"]:
            self.visit_BINOP(ast_node_id, ctx)
        elif cur_type in ["Block", "Start"]:
            self.visit_GENERIC_BLOCK(ast_node_id, ctx)
        elif cur_type in ["PLACEHOLDER", "StatementBody"]: # Node to ignore
            self.visit_passthrough(ast_node_id, ctx)
        elif cur_type in ["FunctionCall"]:
            self.visit_FUNC_CALL(ast_node_id, ctx)
        elif cur_type in ["ArgumentList"]:
            self.visit_ARG_LIST(ast_node_id, ctx)
        elif cur_type in ["IfThenElseStatement"]:
            self.visit_IFTHENELSE(ast_node_id, ctx)
        elif cur_type in ["Condition"]:
            self.visit_CONDITION(ast_node_id, ctx)
        elif cur_type in ["While"]:
            self.visit_WHILE(ast_node_id, ctx)

        else:
            self.visit_GENERIC(ast_node_id, ctx)

    def visit_passthrough(self, ast_node_id: int, ctx: dict):
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, ctx)



ast_data1 = ASTtoCFGVisitor()
ast_data1.visit(ast_data)
ast_data1.cfg.show()


Visit AST from file code.php


## While Avec Break

Pour traiter ce cas nous avons ajouté le __visitINTERRUP__ pour traiter le cas du break et du continue et __visitIFTHEN__ pour le cas sans le else
- __visitINTERRUP__ : dans ce cas là on enlève le ctx[\"parent\"] car les instructions **break** et **continue** n'ont aucune enfants
- __visitIFTHEN__ : dans ce cas là on ajoute le noeud "IfEnd" comme seconde enfant de la condition.

Dans ce dernier cas nous avons des noeuds **Arguement** qui sont déjà ajouté, nous décidons de les ignorer


In [96]:
from code_analysis import ASTReader, AST, CFG, CFGException, ASTException

reader = ASTReader()
f_path = '../part_1/while/while_continue_break.php.ast.json'
ast_data = reader.read_ast(f_path)

class ASTtoCFGVisitor:
    def __init__(self):
        self.ast = None
        self.cfg = CFG()
        self.iNextNode = 0

    def get_new_node(self) -> int:
        self.iNextNode += 1
        return self.iNextNode

    def visit(self, ast: AST):
        self.ast = ast
        self.cfg = CFG()
        print(f"Visit AST from file {self.ast.get_filename()}")
        self.visit_ROOT()
        return self.cfg

    def visit_ROOT(self):
        ctx = {}
        entryNodeId = self.get_new_node()
        stopNodeId = self.get_new_node()
        rootAST = self.ast.get_root()
        self.cfg.set_root(entryNodeId)

        self.cfg.set_type(entryNodeId, "Entry")
        self.cfg.set_image(entryNodeId, "main")
        self.cfg.set_type(stopNodeId, "Exit")

        ctx['parent'] = entryNodeId
        ctx['scope'] = entryNodeId
        ctx['stopId'] = stopNodeId

        if self.ast.get_type(rootAST) == "Start":
            self.cfg.set_node_ptr(rootAST, entryNodeId)

        self.visit_node(rootAST, ctx)
        self.cfg.add_edge(ctx['endId'], stopNodeId)

    # chain nodes
    def visit_GENERIC(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)

        ctx["endId"] = cfg_node

        new_ctx = dict(ctx) # clone ctx
        new_ctx["parent"] = cfg_node
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
        ctx["endId"] = new_ctx["endId"]

        return cfg_node

    def visit_GENERIC_BLOCK(self, ast_node_id: int, ctx: dict):
        new_ctx = dict(ctx) # clone ctx
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
        ctx["endId"] = new_ctx["endId"]

        return None

    def visit_BINOP(self, ast_node_id: int, ctx: dict) -> int:
        #Create BinOP node
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))

        if self.ast.get_type(ast_node_id) == "BinOP" and self.ast.get_image(ast_node_id) == "=":
            right_child = self.ast.get_children(ast_node_id)[1]
            left_child = self.ast.get_children(ast_node_id)[0]
        else:
            right_child = self.ast.get_children(ast_node_id)[0]
            left_child = self.ast.get_children(ast_node_id)[1]
            
        #Visit right child
        new_ctx = dict(ctx) # clone ctx
        self.visit_node(right_child, new_ctx)
        right = new_ctx['endId']

        #Visit right left
        new_ctx = dict(ctx) # clone ctx
        new_ctx["parent"] = right
        self.visit_node(left_child, new_ctx)
        left = new_ctx['endId']

        #Link left child with BinOp
        self.cfg.add_edge(left, cfg_node)
        ctx["endId"] = cfg_node

        
        if self.cfg.get_type(cfg_node) == "BinOP" and self.ast.get_image(ast_node_id) == "=":
            self.cfg.set_op_hands(cfg_node, left, right)
        else:
            self.cfg.add_call_arg(cfg_node, right)
            self.cfg.add_call_arg(cfg_node, left)

        return cfg_node

    def visit_FUNC_CALL(self, ast_node_id: int, ctx: dict) -> int:
         #Create CallBegin node
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)




        # Ajout du noeud CallBegin
        cfg_callbegin_node = self.get_new_node()
        self.cfg.set_type(cfg_callbegin_node, "CallBegin")
        self.cfg.set_image(cfg_callbegin_node, self.ast.get_image(ast_node_id))

        #Ajout du noeud CallEnd
        cfg_callend_node = self.get_new_node()
        self.cfg.set_type(cfg_callend_node, "CallEnd")
        self.cfg.set_image(cfg_callend_node, self.ast.get_image(ast_node_id))

        #Flèche pointillé représentant l'appel de la fonction
        self.cfg.set_call(cfg_callbegin_node, cfg_callend_node)
        


        ctx["endId"] = cfg_callbegin_node
        ctx["parent"] = cfg_node
        ctx["Callbegin"] = cfg_callbegin_node

        new_ctx = dict(ctx)
        #Visite des enfants pour les lier avant l'appel de la fonction
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
            #Si l'enfant est un Id, on le lie au CallBegin
            if self.ast.get_type(child_id) == "Id" and self.ast.get_image(child_id) == self.ast.get_image(ast_node_id):
                self.cfg.set_call_expr(cfg_callbegin_node, new_ctx["endId"])



        # Ajout du noeud RetValue
        cfg_retvalue_node = self.get_new_node()
        self.cfg.set_type(cfg_retvalue_node, "RetValue")
        #Flèche entre CallEnd et RetValue
        self.cfg.add_edge(cfg_callend_node, cfg_retvalue_node)

                
                         
        
        #Flèche entre la fin des enfant et argument
        self.cfg.add_edge(new_ctx["endId"], ctx["endId"])
        ctx["endId"] = cfg_retvalue_node


    def visit_ARG_LIST(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)


        ctx["parent"] = cfg_node

        new_ctx = dict(ctx)
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
            if "Callbegin" in ctx and  self.cfg.get_type(ctx["Callbegin"]) == "CallBegin":
                self.cfg.add_call_arg(ctx["Callbegin"], new_ctx["endId"])


        # Ajout du noeud argument
        cfg_argument_node = self.get_new_node()
        self.cfg.set_type(cfg_argument_node, "Argument")
        ctx["endId"] = cfg_argument_node
        self.cfg.add_edge(new_ctx["endId"], ctx["endId"])


    def visit_IFTHENELSE(self, ast_node_id: int, ctx: dict) -> int:
        #Moddification du noeud IfThenElseStatement
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, "If")
        self.cfg.add_edge(ctx["parent"], cfg_node)

        #Ajout du noeud IfEnd
        cfg_ifend_node = self.get_new_node()
        self.cfg.set_type(cfg_ifend_node, "IfEnd")
        ctx["endId"] = cfg_ifend_node

        ctx["parent"] = cfg_node

        children = self.ast.get_children(ast_node_id)
        new_ctx = dict (ctx)
        self.visit_node(children[0], new_ctx)
        ctx ["parent"] = new_ctx ["endId"]
        #Parcours des enfants
        new_ctx = dict(ctx) # clone ctx
        for child_id in children[1:]:
            self.visit_node(child_id, new_ctx)
            self.cfg.add_edge(new_ctx["endId"], ctx["endId"])

    def visit_CONDITION(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))


        children = self.ast.get_children(ast_node_id)
        new_ctx = dict(ctx)

    
        self.visit_node(children[0], new_ctx)

        self.cfg.add_edge(new_ctx["endId"], cfg_node)

        ctx["endId"] = cfg_node

    def visit_WHILE(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)

        #Ajout du noeud WhileEnd
        cfg_whileend_node = self.get_new_node()
        self.cfg.set_type(cfg_whileend_node, "WhileEnd")
        ctx["endId"] = cfg_whileend_node

        ctx["parent"] = cfg_node
        ctx["whileend"] = cfg_whileend_node
        ctx["while"] = cfg_node

        new_ctx = dict(ctx)
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            new_ctx["parent"] = new_ctx["endId"]
            if self.ast.get_type(child_id) == "Condition":
                self.cfg.add_edge(new_ctx["endId"], ctx["endId"])

        self.cfg.add_edge(new_ctx["endId"], cfg_node)
        
    def visit_INTERRUP(self, ast_node_id: int, ctx: dict) -> int:
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, self.ast.get_type(ast_node_id))
        self.cfg.set_image(cfg_node, self.ast.get_image(ast_node_id))
        self.cfg.add_edge(ctx["parent"], cfg_node)

        # Après un continue ou un break les instructions ne sont pas exécutées

        ctx["endId"] = 0


        new_ctx = dict(ctx) # clone ctx
        
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, new_ctx)
            #new_ctx["parent"] = new_ctx["endId"]


        if self.ast.get_type(ast_node_id) == "Continue":
            self.cfg.add_edge(cfg_node, ctx["while"])
        elif self.ast.get_type(ast_node_id) == "Break":
            self.cfg.add_edge(cfg_node, ctx["whileend"])

    def visit_IFTHEN(self, ast_node_id: int, ctx: dict):
        #Moddification du noeud IfThenStatement
        cfg_node = self.get_new_node()
        self.cfg.set_node_ptr(ast_node_id, cfg_node)
        self.cfg.set_type(cfg_node, "If")
        self.cfg.add_edge(ctx["parent"], cfg_node)

        #Ajout du noeud IfEnd
        cfg_ifend_node = self.get_new_node()
        self.cfg.set_type(cfg_ifend_node, "IfEnd")
        ctx["endId"] = cfg_ifend_node

        ctx["parent"] = cfg_node

        #cette fois-ci on lie la condition à IfEnd
        children = self.ast.get_children(ast_node_id)
        new_ctx = dict (ctx)
        self.visit_node(children[0], new_ctx)
        ctx ["parent"] = new_ctx ["endId"]
        self.cfg.add_edge(new_ctx["endId"], cfg_ifend_node)
        #Parcours des enfants
        new_ctx = dict(ctx) # clone ctx
        for child_id in children[1:]:
            self.visit_node(child_id, new_ctx)
            self.cfg.add_edge(new_ctx["endId"], ctx["endId"])
            print(self.ast.get_type(child_id))

        

    def visit_node(self, ast_node_id: int, ctx: dict):
        cur_type = self.ast.get_type(ast_node_id)
        
        if cur_type is None:
            raise ASTException("Missing type in a node")

        if cur_type in ["BinOP", "RelOP", "LogicOP"]:
            self.visit_BINOP(ast_node_id, ctx)
        elif cur_type in ["Block", "Start"]:
            self.visit_GENERIC_BLOCK(ast_node_id, ctx)
        elif cur_type in ["PLACEHOLDER", "StatementBody","Argument"]: # Node to ignore
            self.visit_passthrough(ast_node_id, ctx)
        elif cur_type in ["FunctionCall"]:
            self.visit_FUNC_CALL(ast_node_id, ctx)
        elif cur_type in ["ArgumentList"]:
            self.visit_ARG_LIST(ast_node_id, ctx)
        elif cur_type in ["IfThenElseStatement"]:
            self.visit_IFTHENELSE(ast_node_id, ctx)
        elif cur_type in ["Condition"]:
            self.visit_CONDITION(ast_node_id, ctx)
        elif cur_type in ["Continue","Break"]:
            self.visit_INTERRUP(ast_node_id, ctx)
        elif cur_type in ["While"]:
            self.visit_WHILE(ast_node_id, ctx)
        elif cur_type in ["IfThenStatement"]:
            self.visit_IFTHEN(ast_node_id, ctx)

        else:
            self.visit_GENERIC(ast_node_id, ctx)

    def visit_passthrough(self, ast_node_id: int, ctx: dict):
        for child_id in self.ast.get_children(ast_node_id):
            self.visit_node(child_id, ctx)



ast_data1 = ASTtoCFGVisitor()
ast_data1.visit(ast_data)
ast_data1.cfg.show()


Visit AST from file code.php
StatementBody


# Partie 2
